Прогнозирование акций Лукойл

In [2]:
#Загружаем библиотеки

import pandas as pd #Пандас
import matplotlib.pyplot as plt #Отрисовка графиков
from tensorflow.keras import utils #Для to_categorical
import numpy as np #Numpy
from keras.optimizers import Adam #Оптимизатор
from keras.models import Sequential, Model #Два варианты моделей
from keras.layers import concatenate, Input, Dense, Dropout, BatchNormalization, Flatten, Conv1D, Conv2D, LSTM #Стандартные слои
from sklearn.preprocessing import StandardScaler, MinMaxScaler #Нормировщики

# Рисовать графики сразу же
%matplotlib inline

import os
print ("Ok")

Ok


In [33]:
#Загружаем датафрейм
frame = pd.read_csv('Gosloto_6x45_20200927.csv', header=0, sep=',')

In [34]:
# удвляем дату и тираж
frame = frame.drop(columns=['datetime', 'tiraz'], axis=1).astype(float)

In [31]:
ax1 = frame.plot.scatter(x='length',

                      y='width',

                      c='DarkBlue')

KeyError: "['datetime' 'tiraz'] not found in axis"